In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import numpy as np
import pandas as pd
import cv2 
import matplotlib.pyplot as plt
from skimage.feature import graycomatrix, graycoprops

In [2]:
import os, glob, cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import get_custom_objects
#import efficientnet.tfkeras as efn
from tqdm import tqdm
import fnmatch

In [3]:
SEED = 42
EPOCHS = 100
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224

In [7]:
IMG_ROOT = './cataractdataset/dataset/'
IMG_DIR = [IMG_ROOT+'1_normal', 
           IMG_ROOT+'2_cataract', 
           IMG_ROOT+'2_glaucoma', 
           IMG_ROOT+'3_retina_disease']

In [8]:
#ocular -disease-recognition dataset
OCU_IMG_ROOT = './Ocular disease Recognition/preprocessed_images/'
ocu_df = pd.read_csv('./Ocular disease Recognition/full_df.csv')

In [9]:
len(ocu_df)

6392

In [10]:
#Cataract dataset Process

cat_df = pd.DataFrame(0, 
                  columns=['paths', 
                           'cataract'],
                  index=range(601))


filepaths = []
for root, dirnames, filenames in os.walk(IMG_ROOT):
    for filename in fnmatch.filter(filenames, '*.png'):
        filepaths.append(os.path.join(root, filename))

In [11]:
filepaths

['./cataractdataset/dataset/1_normal\\NL_001.png',
 './cataractdataset/dataset/1_normal\\NL_002.png',
 './cataractdataset/dataset/1_normal\\NL_003.png',
 './cataractdataset/dataset/1_normal\\NL_004.png',
 './cataractdataset/dataset/1_normal\\NL_005.png',
 './cataractdataset/dataset/1_normal\\NL_006.png',
 './cataractdataset/dataset/1_normal\\NL_007.png',
 './cataractdataset/dataset/1_normal\\NL_008.png',
 './cataractdataset/dataset/1_normal\\NL_009.png',
 './cataractdataset/dataset/1_normal\\NL_010.png',
 './cataractdataset/dataset/1_normal\\NL_011.png',
 './cataractdataset/dataset/1_normal\\NL_012.png',
 './cataractdataset/dataset/1_normal\\NL_013.png',
 './cataractdataset/dataset/1_normal\\NL_014.png',
 './cataractdataset/dataset/1_normal\\NL_015.png',
 './cataractdataset/dataset/1_normal\\NL_016.png',
 './cataractdataset/dataset/1_normal\\NL_017.png',
 './cataractdataset/dataset/1_normal\\NL_018.png',
 './cataractdataset/dataset/1_normal\\NL_019.png',
 './cataractdataset/dataset/1_n

In [12]:
for i, filepath in enumerate(filepaths):
    filepath = os.path.split(filepath)
    cat_df.iloc[i, 0] = filepath[0] + '/' + filepath[1]
    if filepath[0] == IMG_DIR[0]:    # normal
        cat_df.iloc[i, 1] = 0
    elif filepath[0] == IMG_DIR[1]:  # cataract
        cat_df.iloc[i, 1] = 1
    elif filepath[0] == IMG_DIR[2]:  # glaucoma
        cat_df.iloc[i, 1] = 2
    elif filepath[0] == IMG_DIR[3]:  # retine_disease
        cat_df.iloc[i, 1] = 3
        

In [13]:
len(cat_df)

601

In [14]:
cat_df = cat_df.query('0 <= cataract < 2')
len(cat_df)

400

In [15]:
print('Number of normal and cataract images')
print(cat_df['cataract'].value_counts())

Number of normal and cataract images
0    300
1    100
Name: cataract, dtype: int64


In [16]:
def has_cataract_mentioned(text):
    if 'cataract' in text:
        return 1
    elif 'normal' in text:
        return 0
    else:
        return 2

In [17]:
len(ocu_df)

6392

In [18]:
ocu_df['left_eye_cataract'] = ocu_df['Left-Diagnostic Keywords'].apply(lambda x: has_cataract_mentioned(x))
ocu_df['right_eye_cataract'] = ocu_df['Right-Diagnostic Keywords'].apply(lambda x: has_cataract_mentioned(x))

In [19]:
ocu_df

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,...,A,H,M,O,filepath,labels,target,filename,left_eye_cataract,right_eye_cataract
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,...,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg,1,0
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,...,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg,0,0
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,...,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg,2,2
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,...,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg,2,2
4,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,...,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6387,4686,63,Male,4686_left.jpg,4686_right.jpg,severe nonproliferative retinopathy,proliferative diabetic retinopathy,0,1,0,...,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4686_left.jpg,2,2
6388,4688,42,Male,4688_left.jpg,4688_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,...,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4688_left.jpg,2,2
6389,4689,54,Male,4689_left.jpg,4689_right.jpg,mild nonproliferative retinopathy,normal fundus,0,1,0,...,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4689_left.jpg,2,0
6390,4690,57,Male,4690_left.jpg,4690_right.jpg,mild nonproliferative retinopathy,mild nonproliferative retinopathy,0,1,0,...,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4690_left.jpg,2,2


In [20]:
len(ocu_df)

6392

In [21]:
ocu_df

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,...,A,H,M,O,filepath,labels,target,filename,left_eye_cataract,right_eye_cataract
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,...,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg,1,0
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,...,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg,0,0
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,...,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg,2,2
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,...,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg,2,2
4,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,...,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6387,4686,63,Male,4686_left.jpg,4686_right.jpg,severe nonproliferative retinopathy,proliferative diabetic retinopathy,0,1,0,...,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4686_left.jpg,2,2
6388,4688,42,Male,4688_left.jpg,4688_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,...,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4688_left.jpg,2,2
6389,4689,54,Male,4689_left.jpg,4689_right.jpg,mild nonproliferative retinopathy,normal fundus,0,1,0,...,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4689_left.jpg,2,0
6390,4690,57,Male,4690_left.jpg,4690_right.jpg,mild nonproliferative retinopathy,mild nonproliferative retinopathy,0,1,0,...,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4690_left.jpg,2,2


In [22]:
le_df = ocu_df.loc[:, ['Left-Fundus', 'left_eye_cataract']]\
        .rename(columns={'left_eye_cataract':'cataract'})
le_df['paths'] = OCU_IMG_ROOT + le_df['Left-Fundus']

re_df = ocu_df.loc[:, ['Right-Fundus', 'right_eye_cataract']]\
.rename(columns={'right_eye_cataract':'cataract'})
re_df['paths'] = OCU_IMG_ROOT + re_df['Right-Fundus']

In [23]:
len(re_df)

6392

In [24]:
len(le_df)

6392

In [25]:
re_df['cataract'].value_counts()

2    3317
0    2785
1     290
Name: cataract, dtype: int64

In [26]:
re_df

,Right-Fundus,cataract,paths
0,0_right.jpg,0,./Ocular disease Recognition/preprocessed_imag...
1,1_right.jpg,0,./Ocular disease Recognition/preprocessed_imag...
2,2_right.jpg,2,./Ocular disease Recognition/preprocessed_imag...
3,4_right.jpg,2,./Ocular disease Recognition/preprocessed_imag...
4,5_right.jpg,2,./Ocular disease Recognition/preprocessed_imag...
...,...,...,...
6387,4686_right.jpg,2,./Ocular disease Recognition/preprocessed_imag...
6388,4688_right.jpg,2,./Ocular disease Recognition/preprocessed_imag...
6389,4689_right.jpg,0,./Ocular disease Recognition/preprocessed_imag...
6390,4690_right.jpg,2,./Ocular disease Recognition/preprocessed_imag...


In [27]:
le_df['cataract'].value_counts()

2    3190
0    2898
1     304
Name: cataract, dtype: int64

In [28]:
print('Number of left eye images')
print(le_df['cataract'].value_counts())
print('\nNumber of right eye images')
print(re_df['cataract'].value_counts())

Number of left eye images
2    3190
0    2898
1     304
Name: cataract, dtype: int64

Number of right eye images
2    3317
0    2785
1     290
Name: cataract, dtype: int64


In [29]:
le_df= le_df.query('0 <= cataract < 2')
re_df= re_df.query('0 <= cataract < 2')

In [30]:
print('Number of left eye images')
print(le_df['cataract'].value_counts())
print('\nNumber of right eye images')
print(re_df['cataract'].value_counts())

Number of left eye images
0    2898
1     304
Name: cataract, dtype: int64

Number of right eye images
0    2785
1     290
Name: cataract, dtype: int64


In [31]:
def downsample(df):
    df = pd.concat([df.query('cataract==1'),
        df.query('cataract==0').sample(sum(df['cataract']), 
                                       random_state=SEED)
    ])
    return df

In [32]:
le_df = downsample(le_df)
re_df = downsample(re_df)

In [33]:
print('Number of left eye images')
print(le_df['cataract'].value_counts())
print('\nNumber of right eye images')
print(re_df['cataract'].value_counts())

Number of left eye images
1    304
0    304
Name: cataract, dtype: int64

Number of right eye images
1    290
0    290
Name: cataract, dtype: int64


In [34]:
re_df = re_df.drop('Right-Fundus', axis=1)
le_df = le_df.drop('Left-Fundus', axis=1)

In [35]:
re_df

,cataract,paths
20,1,./Ocular disease Recognition/preprocessed_imag...
65,1,./Ocular disease Recognition/preprocessed_imag...
93,1,./Ocular disease Recognition/preprocessed_imag...
160,1,./Ocular disease Recognition/preprocessed_imag...
186,1,./Ocular disease Recognition/preprocessed_imag...
...,...,...
4180,0,./Ocular disease Recognition/preprocessed_imag...
2413,0,./Ocular disease Recognition/preprocessed_imag...
5302,0,./Ocular disease Recognition/preprocessed_imag...
5686,0,./Ocular disease Recognition/preprocessed_imag...


In [36]:
ocu_df = pd.concat([le_df, re_df], axis=0)

In [37]:
ocu_df['cataract'].value_counts()

1    594
0    594
Name: cataract, dtype: int64

In [38]:
cat_df['cataract'].value_counts()

0    300
1    100
Name: cataract, dtype: int64

In [39]:
file_name = 'LeftFundus.xlsx'
le_df.to_excel(file_name)


In [40]:
file= 'RightFundus.xlsx'
re_df.to_excel(file)

In [41]:
df = pd.concat([cat_df, ocu_df], ignore_index=True)

In [42]:
df

,paths,cataract
0,./cataractdataset/dataset/1_normal/NL_001.png,0
1,./cataractdataset/dataset/1_normal/NL_002.png,0
2,./cataractdataset/dataset/1_normal/NL_003.png,0
3,./cataractdataset/dataset/1_normal/NL_004.png,0
4,./cataractdataset/dataset/1_normal/NL_005.png,0
...,...,...
1583,./Ocular disease Recognition/preprocessed_imag...,0
1584,./Ocular disease Recognition/preprocessed_imag...,0
1585,./Ocular disease Recognition/preprocessed_imag...,0
1586,./Ocular disease Recognition/preprocessed_imag...,0


In [43]:
df['cataract'].value_counts()

0    894
1    694
Name: cataract, dtype: int64

In [44]:
destinationFolder = "./CombinedCataract/Dataset/"
keys = ["Normal", "Cataract"]
for key in keys:
    if not os.path.exists(destinationFolder+'/'+ key):
        os.makedirs(destinationFolder+'/'+key)


In [47]:
def save_img(data_f):
    n=0
    c=0
    for i, path in enumerate(data_f['paths']):
        img= cv2.imread(path)
        
        lebel= data_f[(data_f['paths']==path)]['cataract']
        
        file_name_no_extension = os.path.splitext(filename)[0]
        #print(file_name_no_extension)
        if lebel[i]== 0:
            cv2.imwrite(destinationFolder+keys[0]+'/'+str(n)+".jpg",img)
            n+=1
            #print(i)
        else:
            cv2.imwrite(destinationFolder+keys[1]+'/'+str(c)+".jpg",img)
            c+=1
            #print(i)
       
                


In [48]:
save_img(df)

In [ ]:
df['paths']

In [ ]:
for i in df['paths']:
    print(i)

In [ ]:
file_name = 'MarksData.xlsx'
df.to_excel(file_name)